In [1]:
import pandas as pd
import numpy
import geopy.distance
import math
import sys

In [2]:
dataset = pd.read_csv("preprocessed_dataset.csv")

In [3]:
# Calculate the distance between the two points
# Return a binary value for match/mismatch


def point_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    if dist < 30:
        return 0
    else:
        return 1

In [4]:
# calculate the distance between the two points


def actual_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    return round(dist)

In [5]:
# Create similarity matrix between two tracks.


def create_matrix(track1, track2):
    matrix = numpy.matrix(numpy.ones((len(track1) + 1, len(track2) + 1)) * numpy.inf)
    matrix[0, 0] = 0

    for i in range(1, len(track1) + 1):
        for j in range(1, len(track2) + 1):
            dist = actual_distance(track1[i - 1], track2[j - 1])
            matrix[i, j] = (
                min(matrix[i - 1, j], min(matrix[i, j - 1], matrix[i - 1, j - 1]))
                + dist
            )

    return matrix

In [6]:
# Use the matrix to backtrack the best alignment between the tracks.
# Follow the lowest numbers until the end of the matrix is reached.


def backtrack(matrix, track1, track2):
    a = []
    i = len(track1)
    j = len(track2)

    while i > 0 and j > 0:

        minimum = min(matrix[i - 1, j - 1], min(matrix[i - 1, j], matrix[i, j - 1]))
        if matrix[i - 1, j] == minimum:
            i = i - 1
        elif matrix[i, j - 1] == minimum:
            j = j - 1
        elif matrix[i - 1, j - 1] == minimum:
            j = j - 1
            i = i - 1
        if (
            point_distance(
                (track1[i - 1][0], track1[i - 1][1]),
                (track2[j - 1][0], track2[j - 1][1]),
            )
            == 0
        ):
            point = [track1[i - 1][0], track1[i - 1][1], track2[0][2]]
            if len(a) == 0 or a[0] != point:
                a.insert(0, point)
    return a

In [7]:
# List of all track ids in the dataset.

track_ids = pd.read_csv("preprocessed_list_of_track_ids.csv").values.tolist()

In [8]:
# This is the implementation if the Dynamic Time Warping algorithm.
# It takes one track as input and compares that track to every other track in the dataset coming after that track.


def Dynamic_Time_Warping(dataset, trackid, nr):
    track = dataset.loc[dataset["id"] == trackid]
    track = pd.DataFrame.drop_duplicates(track)
    print("track1: ", trackid)
    alignments = []
    for j in range(nr + 1, len(track_ids)):
        next_track = dataset.loc[dataset["id"] == track_ids[j][0]]
        matrix = create_matrix(track.values.tolist(), next_track.values.tolist())
        alignment = backtrack(matrix, track.values.tolist(), next_track.values.tolist())
        if len(alignment) > 3:
            alignments.extend(alignment)
    return alignments

In [9]:
# Create an overview of all points in the track and which other tracks has a matching point to that point.
# The output is a list of coordinate points where each has a corresponding list of track-ids.


def create_list_of_points(tracknr, alignments):

    track = dataset.loc[dataset["id"] == tracknr]
    track = pd.DataFrame.drop_duplicates(track)
    points = pd.DataFrame(columns=["lat", "lon", "ids"])
    align = pd.DataFrame(alignments, columns=["lat", "lon", "id"])

    for i in range(len(track)):
        lat = align.loc[align["lat"] == track.iloc[i]["lat"]]
        points.loc[i] = [
            track.iloc[i]["lat"],
            track.iloc[i]["lon"],
            lat["id"].values.tolist(),
        ]
    return points

In [10]:
# Convert the list of tracks-ids to number of tracks.
# The output is a list of coordinate points with a corresponding number of track ids matching with that point.


def convert_id_list_to_number_of_tracks(points):
    point_list = pd.DataFrame(points, columns=["lat", "lon", "ids"])
    for i in range(len(point_list)):
        distinct_point_list = list(set(point_list.loc[i]["ids"]))
        val = len(distinct_point_list)
        point_list.at[i, "ids"] = val
    return point_list

In [11]:
# The list of points from the previous function will be converted to result sequences.
# Each compinination of coordinate points with a high frequency will be awarded a value.
# The value is calculated by adding the length of the sequence, how many coordinate points are included,
# and the frequency of the sequence, the number of tracks matching the entire subtrack.
# The combination of coordinate points acchieving the highest value will be added returned.
# If there are multiple subsequences that are not overlapping acchiving a high value, all will be added to the final_records-list.


def gather_sequences(points):
    i = 0
    final_records = pd.DataFrame(columns=["polyline", "value"])
    for group in range(points.iloc[0]["group"], -1, -1):
        records = pd.DataFrame(columns=["polyline", "value"])
        sequence = []
        value = 0
        while i < len(points) and points.iloc[i]["group"] == group:
            if value == points.iloc[i]["ids"] or value == 0:
                sequence.append(
                    [
                        points.index[i],
                        points.iloc[i]["lat"],
                        points.iloc[i]["lon"],
                    ]
                )
                value = points.iloc[i]["ids"]
                i += 1
            else:
                sequence.sort()
                new_sequence = sequence.copy()
                if len(new_sequence) >= 3:
                    records.loc[len(records)] = [
                        new_sequence,
                        (len(new_sequence) * 1.0) + value,
                    ]
                value = points.iloc[i]["ids"]

        sequence.sort()
        new_sequence = sequence.copy()
        if len(new_sequence) >= 3:
            records.loc[len(records)] = [
                new_sequence,
                (len(new_sequence) * 1.0) + value,
            ]
        if len(records) < 2:
            final_records = pd.concat([final_records, records])
        else:
            highest_value = records["value"].idxmax()
            final_records = final_records.append(records.loc[highest_value])

    return final_records

In [24]:
# This is the fuction combining all the functions above.
# All the other functions are running for all tracks in the dataset.
# The output of this function is the complete result of the most trafficked roads using Smith-Waterman.


def hele_sullamitten(dataset):
    most_tracked_roads = pd.DataFrame(columns=["polyline", "value"])
    for i in range(len(track_ids)):
        track = track_ids[i][0]
        alignment = Dynamic_Time_Warping(dataset, track, i)
        points = create_list_of_points(track, alignment)
        new_points = convert_id_list_to_number_of_tracks(points)
        new_points = new_points[new_points.ids > 8]
        if not new_points.empty:
            new_points["group"] = (new_points.index.to_series().diff() > 2).cumsum()
            new_points.sort_values(by=["group", "ids"], ascending=False, inplace=True)
            records = gather_sequences(new_points)
            most_tracked_roads = pd.concat([most_tracked_roads, records])
            print(most_tracked_roads)
    return most_tracked_roads


res = hele_sullamitten(dataset)

track1:  T1
                                            polyline  value
0  [[2, 41.148855000000005, -8.585685000000002], ...   26.0
track1:  T2
                                            polyline  value
0  [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
track1:  T2334
track1:  T3
                                            polyline  value
0  [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
track1:  T3463
track1:  T3569
track1:  T3985
track1:  T4
track1:  T61199
track1:  T612104
track1:  T7
                                            polyline  value
0  [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3  [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0  [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
track1:

                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
3   [[0, 41.148603, -8.585694], [1, 41.148684, -8....   27.0
1   [[0, 41.150484000000006, -8.592165], [1, 41.15...   20.0
3   [[244, 41.14596, -8.613846], [245, 41.14597499...   31.0
3   [[229, 41.1452145, -8.6164155], [230, 41.14519...   21.0
1   [[219, 41.145993, -8.617905], [220, 41.1458265...   16.0
0   [[189, 41.14765800000001, -8.622324000000003],...   14.0
0   [[160, 41.15010599999998, -8.625807000000005],...   14.0
0   [[152, 41.15148428571428, -8.625922714285712],...   14.0
0   [[131, 41.15245080000002, -8.628520799999999],...   16.0
0   [[118, 41.1525945, -8.6307705], [119, 41.15256...   17.0
0   [[106, 41.1525720000

                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
3   [[0, 41.148603, -8.585694], [1, 41.148684, -8....   27.0
1   [[0, 41.150484000000006, -8.592165], [1, 41.15...   20.0
3   [[244, 41.14596, -8.613846], [245, 41.14597499...   31.0
3   [[229, 41.1452145, -8.6164155], [230, 41.14519...   21.0
1   [[219, 41.145993, -8.617905], [220, 41.1458265...   16.0
0   [[189, 41.14765800000001, -8.622324000000003],...   14.0
0   [[160, 41.15010599999998, -8.625807000000005],...   14.0
0   [[152, 41.15148428571428, -8.625922714285712],...   14.0
0   [[131, 41.15245080000002, -8.628520799999999],...   16.0
0   [[118, 41.1525945, -8.6307705], [119, 41.15256...   17.0
0   [[106, 41.1525720000

track1:  T36541142
track1:  T36551189
track1:  T36571195
track1:  T37
track1:  T37261215
track1:  T38
track1:  T3811240
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
3   [[0, 41.148603, -8.585694], [1, 41.148684, -8....   27.0
1   [[0, 41.150484000000006, -8.592165], [1, 41.15...   20.0
3   [[244, 41.14596, -8.613846], [245, 41.14597499...   31.0
3   [[229, 41.1452145, -8.6164155], [230, 41.14519...   21.0
1   [[219, 41.145993, -8.617905], [220, 41.1458265...   16.0
0   [[189, 41.14765800000001, -8.622324000000003],...   14.0
0   [[160, 41.15010599999998, -8.625807000000005],...   14.0
0   [[152, 41.15148428571428, -8.625922714285712],...   14.0
0   [[131, 41.152450800000

track1:  T46601439
track1:  T46191445
track1:  T47231461
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
3   [[0, 41.148603, -8.585694], [1, 41.148684, -8....   27.0
1   [[0, 41.150484000000006, -8.592165], [1, 41.15...   20.0
3   [[244, 41.14596, -8.613846], [245, 41.14597499...   31.0
3   [[229, 41.1452145, -8.6164155], [230, 41.14519...   21.0
1   [[219, 41.145993, -8.617905], [220, 41.1458265...   16.0
0   [[189, 41.14765800000001, -8.622324000000003],...   14.0
0   [[160, 41.15010599999998, -8.625807000000005],...   14.0
0   [[152, 41.15148428571428, -8.625922714285712],...   14.0
0   [[131, 41.15245080000002, -8.628520799999999],...   16.0
0   [[118, 41.1525945, -8.63

...

                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
..                                                ...    ...
0   [[11, 41.15385, -8.611992], [12, 41.1540462, -...   12.0
15  [[26, 41.147919, -8.611236000000002], [27, 41....  191.0
1   [[7, 41.15401200000001, -8.613279], [8, 41.153...   15.0
0   [[79, 41.169413250000005, -8.606126249999999],...   22.0
3   [[8, 41.157576, -8.60922], [9, 41.157739125, -...   78.0

[76 rows x 2 columns]
track1:  T98232783
track1:  T98662801
track1:  T98672804
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [

                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
..                                                ...    ...
2   [[1, 41.14566000000001, -8.610795], [2, 41.145...   17.0
0   [[31, 41.14612472727271, -8.612690727272721], ...   19.0
1   [[0, 41.145867, -8.610489], [1, 41.14600272000...   17.0
11  [[13, 41.147712, -8.6085], [15, 41.147691, -8....  103.0
0   [[1, 41.146242, -8.617679], [2, 41.146095, -8....   12.0

[90 rows x 2 columns]
track1:  T133213510
track1:  T135
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.58573

track1:  T179814407
track1:  T179174411
track1:  T17984415
track1:  T17904427
track1:  T18014434
track1:  T181
track1:  T181654460
track1:  T181664465
track1:  T181674471
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
..                                                ...    ...
10  [[36, 41.14800514285714, -8.612007428571431], ...  172.0
1   [[17, 41.147433, -8.608806000000001], [18, 41....   26.0
0   [[4, 41.151672000000005, -8.609634000000002], ...   24.0
1   [[3, 41.147676, -8.620217999999998], [4, 41.14...   36.0
0   [[7, 41.153841, -8.61327], [8, 41.153841000000...   16.0

[107 rows x 2 columns]
track1:  T1811024490
track1:  T1811084525
track1:  T181414554
track1:  T1

                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
..                                                ...    ...
0   [[3, 41.164371, -8.606501999999999], [4, 41.16...   16.0
1   [[87, 41.1475094117647, -8.620210058823556], [...   28.0
1   [[71, 41.14766752941176, -8.617304647058841], ...   16.0
1   [[52, 41.14785529411765, -8.613854470588242], ...   15.0
0   [[42, 41.14795411764706, -8.612038588235295], ...   14.0

[129 rows x 2 columns]
track1:  T222
track1:  T22215997
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.58573

                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.145...   30.0
0   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
2   [[0, 41.148576000000006, -8.585658], [1, 41.14...   27.0
..                                                ...    ...
4   [[29, 41.147778, -8.620254], [30, 41.147716, -...   92.0
0   [[10, 41.147487, -8.617914000000003], [11, 41....   14.0
1   [[95, 41.152482, -8.628488999999995], [96, 41....   19.0
2   [[27, 41.14779525, -8.619977250000002], [28, 4...   74.0
0   [[10, 41.14745228571429, -8.617955142857143], ...   12.0

[144 rows x 2 columns]
track1:  T25037152
track1:  T250347159
track1:  T2511197177
track1:  T252
                                             polyline  value
0   [[2, 41.148855000000005, -8.585685000000002], ...   26.0
3   [[0, 41.14557, -8.610876000000001], [1, 41.1

In [25]:
pd.DataFrame.to_csv(res, "./DTW_result.csv", index=True)